# Just to casually look at and try things

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

from sklearn.metrics import mean_squared_error as mse

In [3]:
Spok = pd.read_csv('NOAA_Spokane_Airport.csv')

In [4]:
Spok

,,,,,,()
Date,TAVG (Degrees Fahrenheit),TMAX (Degrees Fahrenheit),TMIN (Degrees Fahrenheit),PRCP (Inches),SNOW (Inches),SNWD (Inches)


In [5]:
Spok.head(5)

,,,,,,()
Date,TAVG (Degrees Fahrenheit),TMAX (Degrees Fahrenheit),TMIN (Degrees Fahrenheit),PRCP (Inches),SNOW (Inches),SNWD (Inches)


In [4]:
y_holdout = np.array([1, 1, 2, 3])
pred_lin = np.array([0, 1, 1, 12])



np.sqrt(np.mean(y_holdout-pred_lin)**2), mse(y_holdout, pred_lin)

(1.75, 20.75)

# model functions
I want to test some of my functions for the model I'm building, particularly the data rearranging and recursive parts, so let's generate a dataframe

I need to figure out what to do for data prep, since I don't think I can use Marcos's `prep_data_for_training` as is; I do want to have the target as a column, even if it's one I replace bits of as I go AND I want my first "row" to be the last however long of the training/validation set and the first label to be the first element of the test set

In [98]:
# making slight changes since I don't think it needs to return the labels
WINDOW_SIZE=6

def df_to_X_multi(df, window_size=WINDOW_SIZE):
    df_as_np = df.to_numpy()
    X = []
    for i in range(len(df_as_np)-window_size):
        row = [a for a in df_as_np[i:i+window_size]]
        X.append(row)
    return np.array(X)

#modified slightly so it's easy to keep track of things
def newrow(row, pred):
    row[-1,0] = pred #(pred - well_training_mean) / well_training_std
    return row

In [112]:
# make a dummy dataframe
df = pd.DataFrame(data={"avg_well_depth":[i for i in range(1,27)], 
                        'big':[200+i for i in range(1,27)], 
                        'neg':[-i-200 for i in range(1,27)]})

date_rng = pd.date_range(start='2023-10-21', end='2023-11-15', freq='D')
df['date'] = date_rng
df.head()

,avg_well_depth,big,neg,date
0,1,201,-201,2023-10-21
1,2,202,-202,2023-10-22
2,3,203,-203,2023-10-23
3,4,204,-204,2023-10-24
4,5,205,-205,2023-10-25


In [103]:
import sys
sys.path.insert(0, '../')
from gw_tools.gw_plot import gw_plot
from gw_tools import data_prep

In [113]:
df = data_prep.add_toy_signal(df)
df.head()

,avg_well_depth,big,neg,date,year_sin,year_cos
0,1,201,-201,2023-10-21,-0.945860,0.324575
1,2,202,-202,2023-10-22,-0.940137,0.340797
2,3,203,-203,2023-10-23,-0.934135,0.356919
3,4,204,-204,2023-10-24,-0.927857,0.372935
4,5,205,-205,2023-10-25,-0.921305,0.388841


In [130]:
df_test = df[-15:].copy()
df_train = df[:-15].copy()
df_test, df_train

(    avg_well_depth  big  neg       date  year_sin  year_cos
 11              12  212 -212 2023-11-01 -0.867922  0.496700
 12              13  213 -213 2023-11-02 -0.859250  0.511557
 13              14  214 -214 2023-11-03 -0.850323  0.526262
 14              15  215 -215 2023-11-04 -0.841144  0.540811
 15              16  216 -216 2023-11-05 -0.831717  0.555200
 16              17  217 -217 2023-11-06 -0.822043  0.569425
 17              18  218 -218 2023-11-07 -0.812126  0.583482
 18              19  219 -219 2023-11-08 -0.801969  0.597366
 19              20  220 -220 2023-11-09 -0.791575  0.611073
 20              21  221 -221 2023-11-10 -0.780946  0.624599
 21              22  222 -222 2023-11-11 -0.770086  0.637940
 22              23  223 -223 2023-11-12 -0.758998  0.651093
 23              24  224 -224 2023-11-13 -0.747686  0.664053
 24              25  225 -225 2023-11-14 -0.736152  0.676816
 25              26  226 -226 2023-11-15 -0.724401  0.689379,
     avg_well_depth  bi

In [131]:
test_dates = pd.to_datetime(df_test['date'], format='%d.%m.%Y %H:%M:%S')
train_dates = pd.to_datetime(df_train['date'], format='%d.%m.%Y %H:%M:%S')
for dataf in [df_test, test_dates, df_train, train_dates]:
    display(dataf.head(2))

,avg_well_depth,big,neg,date,year_sin,year_cos
11,12,212,-212,2023-11-01,-0.867922,0.496700
12,13,213,-213,2023-11-02,-0.859250,0.511557


11   2023-11-01
12   2023-11-02
Name: date, dtype: datetime64[ns]

,avg_well_depth,big,neg,date,year_sin,year_cos
0,1,201,-201,2023-10-21,-0.945860,0.324575
1,2,202,-202,2023-10-22,-0.940137,0.340797


0   2023-10-21
1   2023-10-22
Name: date, dtype: datetime64[ns]

In [154]:
fill_preds = pd.DataFrame({'Predictions':df_train.avg_well_depth[-WINDOW_SIZE:], 
                           'date':df_train.date[-WINDOW_SIZE:]})

actual_test = df[-15-WINDOW_SIZE:].copy().merge(fill_preds, how='outer', on='date')
actual_test.drop(columns=['avg_well_depth','date'], inplace=True)
actual_test

,big,neg,year_sin,year_cos,Predictions
0,206,-206,-0.914480,0.404632,6.0
1,207,-207,-0.907384,0.420303,7.0
2,208,-208,-0.900020,0.435849,8.0
3,209,-209,-0.892389,0.451267,9.0
4,210,-210,-0.884494,0.466551,10.0
5,211,-211,-0.876338,0.481697,11.0
6,212,-212,-0.867922,0.496700,NaN
7,213,-213,-0.859250,0.511557,NaN
8,214,-214,-0.850323,0.526262,NaN
9,215,-215,-0.841144,0.540811,NaN


# ignore everything under this for now

the new plan is to make each "row" from the dataframe, do the "prediction," insert the prediction into the new spot in the dataframe using something like
```
actual_test.loc[index,'Prediction']=pred
```
and then loop

This should give me a column of predictions, the first `WINDOW_SIZE` of which are the real data from the validation set and the last however many are real predictions. I'll have to make a df with the actuals and the predictions. This makes me think I can also maybe use marcos's function since I'm going to pop the actuals off anyway? But I do want the date soooo.... we'll see

# ok back and making an attempt

I've decided I think it's better to make the date sine and cosine and then set the date column to be the INDEX so I keep that instead of splitting it off, so let's begin again

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

from sklearn.metrics import mean_squared_error as mse

import sys
sys.path.insert(0, '../')
from gw_tools.gw_plot import gw_plot
from gw_tools import data_prep

WINDOW_SIZE = 6

def df_to_X_multi(df, window_size=WINDOW_SIZE):
    df_as_np = df.to_numpy()
    X = []
    for i in range(len(df_as_np)-window_size):
        row = [a for a in df_as_np[i:i+window_size]]
        X.append(row)
    return np.array(X)

In [61]:
#dummy df

# make a dummy dataframe
df = pd.DataFrame(data={"avg_well_depth":[i for i in range(1,27)], 
                        'big':[200+i for i in range(1,27)], 
                        'neg':[-i-200 for i in range(1,27)]})

date_rng = pd.date_range(start='2023-10-21', end='2023-11-15', freq='D')
df['date'] = date_rng
df.head()

,avg_well_depth,big,neg,date
0,1,201,-201,2023-10-21
1,2,202,-202,2023-10-22
2,3,203,-203,2023-10-23
3,4,204,-204,2023-10-24
4,5,205,-205,2023-10-25


In [62]:
df = data_prep.add_toy_signal(df).set_index('date')

df.head()

,avg_well_depth,big,neg,year_sin,year_cos
date,,,,,
2023-10-21,1,201,-201,-0.945860,0.324575
2023-10-22,2,202,-202,-0.940137,0.340797
2023-10-23,3,203,-203,-0.934135,0.356919
2023-10-24,4,204,-204,-0.927857,0.372935
2023-10-25,5,205,-205,-0.921305,0.388841


In [51]:
df = df.set_index('date')
df.head(2)

,avg_well_depth,big,neg
date,,,
2023-10-21,1,201,-201
2023-10-22,2,202,-202


In [52]:
TEST_SIZE = 15 # size of the test set

df_test = df[-TEST_SIZE:].copy()
df_train = df[:-TEST_SIZE].copy()
df_test.head(2), df_train.head(2)

(            avg_well_depth  big  neg
 date                                
 2023-11-01              12  212 -212
 2023-11-02              13  213 -213,
             avg_well_depth  big  neg
 date                                
 2023-10-21               1  201 -201
 2023-10-22               2  202 -202)

In [53]:
fill_preds = pd.DataFrame({'Predictions':df_train.avg_well_depth[-WINDOW_SIZE:]})

actual_test = df[-TEST_SIZE-WINDOW_SIZE:].copy().merge(fill_preds, how='outer', on='date')
actual_test.drop(columns=['avg_well_depth'], inplace=True)
actual_test

,big,neg,Predictions
date,,,
2023-10-26,206,-206,6.0
2023-10-27,207,-207,7.0
2023-10-28,208,-208,8.0
2023-10-29,209,-209,9.0
2023-10-30,210,-210,10.0
2023-10-31,211,-211,11.0
2023-11-01,212,-212,NaN
2023-11-02,213,-213,NaN
2023-11-03,214,-214,NaN


In [54]:
for i in range(TEST_SIZE):
    row = df_to_X_multi(actual_test[i:i+WINDOW_SIZE+1])
    print(row)
    
    # make prediction and insert in the right place
    pred = i+1000
    pred_date = actual_test.index[WINDOW_SIZE]+pd.Timedelta(days=i)
    actual_test.loc[pred_date,'Predictions']=pred

[[[ 206. -206.    6.]
  [ 207. -207.    7.]
  [ 208. -208.    8.]
  [ 209. -209.    9.]
  [ 210. -210.   10.]
  [ 211. -211.   11.]]]
[[[ 207. -207.    7.]
  [ 208. -208.    8.]
  [ 209. -209.    9.]
  [ 210. -210.   10.]
  [ 211. -211.   11.]
  [ 212. -212. 1000.]]]
[[[ 208. -208.    8.]
  [ 209. -209.    9.]
  [ 210. -210.   10.]
  [ 211. -211.   11.]
  [ 212. -212. 1000.]
  [ 213. -213. 1001.]]]
[[[ 209. -209.    9.]
  [ 210. -210.   10.]
  [ 211. -211.   11.]
  [ 212. -212. 1000.]
  [ 213. -213. 1001.]
  [ 214. -214. 1002.]]]
[[[ 210. -210.   10.]
  [ 211. -211.   11.]
  [ 212. -212. 1000.]
  [ 213. -213. 1001.]
  [ 214. -214. 1002.]
  [ 215. -215. 1003.]]]
[[[ 211. -211.   11.]
  [ 212. -212. 1000.]
  [ 213. -213. 1001.]
  [ 214. -214. 1002.]
  [ 215. -215. 1003.]
  [ 216. -216. 1004.]]]
[[[ 212. -212. 1000.]
  [ 213. -213. 1001.]
  [ 214. -214. 1002.]
  [ 215. -215. 1003.]
  [ 216. -216. 1004.]
  [ 217. -217. 1005.]]]
[[[ 213. -213. 1001.]
  [ 214. -214. 1002.]
  [ 215. -215. 100

In [48]:
actual_test

,big,neg,year_sin,year_cos,Predictions
date,,,,,
2023-10-26,206,-206,-0.914480,0.404632,6.0
2023-10-27,207,-207,-0.907384,0.420303,7.0
2023-10-28,208,-208,-0.900020,0.435849,8.0
2023-10-29,209,-209,-0.892389,0.451267,9.0
2023-10-30,210,-210,-0.884494,0.466551,10.0
2023-10-31,211,-211,-0.876338,0.481697,11.0
2023-11-01,212,-212,-0.867922,0.496700,1000.0
2023-11-02,213,-213,-0.859250,0.511557,1001.0
2023-11-03,214,-214,-0.850323,0.526262,1002.0


In [58]:
results = df_test[['avg_well_depth']].merge(actual_test[["Predictions"]], on='date')
results

,avg_well_depth,Predictions
date,,
2023-11-01,12,1000.0
2023-11-02,13,1001.0
2023-11-03,14,1002.0
2023-11-04,15,1003.0
2023-11-05,16,1004.0
2023-11-06,17,1005.0
2023-11-07,18,1006.0
2023-11-08,19,1007.0
2023-11-09,20,1008.0


In [63]:
np.mean(df.year_sin)

-0.8477206341081405

## old stuff

In [150]:
actual_test = actual_test.merge(fill_preds, how='outer', on='date')
actual_test

,avg_well_depth,big,neg,date,year_sin,year_cos,Predictions
0,6,206,-206,2023-10-26,-0.914480,0.404632,6.0
1,7,207,-207,2023-10-27,-0.907384,0.420303,7.0
2,8,208,-208,2023-10-28,-0.900020,0.435849,8.0
3,9,209,-209,2023-10-29,-0.892389,0.451267,9.0
4,10,210,-210,2023-10-30,-0.884494,0.466551,10.0
5,11,211,-211,2023-10-31,-0.876338,0.481697,11.0
6,12,212,-212,2023-11-01,-0.867922,0.496700,NaN
7,13,213,-213,2023-11-02,-0.859250,0.511557,NaN
8,14,214,-214,2023-11-03,-0.850323,0.526262,NaN
9,15,215,-215,2023-11-04,-0.841144,0.540811,NaN


In [100]:
# prep (similar to what Marcos does in data_prep.py, but I'll have do do a different version I think)

# the predictions column the first window size of them will need to come from the training set somehow
# maybe do a join with 
dates = pd.to_datetime(df.pop('date'), format='%d.%m.%Y %H:%M:%S').copy()
target = df.pop('nums').copy()
features = df.copy().join(pd.DataFrame({'Predictions':target[0:WINDOW_SIZE]}))
features.head(10)

,big,neg,Predictions
0,201,-201,1.0
1,202,-202,2.0
2,203,-203,3.0
3,204,-204,4.0
4,205,-205,5.0
5,206,-206,6.0
6,207,-207,NaN
7,208,-208,NaN
8,209,-209,NaN
9,210,-210,NaN


In [90]:
target[0:7]

0    1
1    2
2    3
3    4
4    5
5    6
6    7
Name: nums, dtype: int64

In [65]:
X[0], X[0][-1]

(array([[   1,  201, -201],
        [   2,  202, -202],
        [   3,  203, -203],
        [   4,  204, -204],
        [   5,  205, -205],
        [   6,  206, -206]]),
 array([   6,  206, -206]))

In [68]:
x0=X[0].copy()
x0

array([[   1,  201, -201],
       [   2,  202, -202],
       [   3,  203, -203],
       [   4,  204, -204],
       [   5,  205, -205],
       [   6,  206, -206]])

AttributeError: 'numpy.ndarray' object has no attribute 'pop'

In [86]:
test = pd.DataFrame({'test':[5,4,3,2,1]})
test

,test
0,5
1,4
2,3
3,2
4,1


In [87]:
features = features.join(test)

In [88]:
features.head(7)

,nums,big,neg,test
0,1,201,-201,5.0
1,2,202,-202,4.0
2,3,203,-203,3.0
3,4,204,-204,2.0
4,5,205,-205,1.0
5,6,206,-206,NaN
6,7,207,-207,NaN


In [89]:
features.loc[5,'test']=18
features.head(7)

,nums,big,neg,test
0,1,201,-201,5.0
1,2,202,-202,4.0
2,3,203,-203,3.0
3,4,204,-204,2.0
4,5,205,-205,1.0
5,6,206,-206,18.0
6,7,207,-207,NaN


In [64]:
df_to_X_multi(features[0:7])

array([[[   1,  201, -201],
        [   2,  202, -202],
        [   3,  203, -203],
        [   4,  204, -204],
        [   5,  205, -205],
        [   6,  206, -206]]])

In [67]:
# "predict", including printing things
row = X[0]
recur_preds = []

for i in range(1,5): #since the size of the test set
    print("start loop",i)
    row = row.reshape(1,6,3)
    print("row\n",row)
    
    # make and store prediction
    pred = i+1000 #an obvious change
    recur_preds.append(pred)
    print("after 'prediction'")
    print(row, pred)
    print()
    
    # scale prediction and insert into the next row
    row = newrow(X[i], pred)
    print(row, pred)
    print()
    print()

start loop 1
row
 [[[   1  201 -201]
  [   2  202 -202]
  [   3  203 -203]
  [   4  204 -204]
  [   5  205 -205]
  [   6  206 -206]]]
after 'prediction'
[[[   1  201 -201]
  [   2  202 -202]
  [   3  203 -203]
  [   4  204 -204]
  [   5  205 -205]
  [   6  206 -206]]] 1001

[[   2  202 -202]
 [   3  203 -203]
 [   4  204 -204]
 [   5  205 -205]
 [   6  206 -206]
 [1001  207 -207]] 1001


start loop 2
row
 [[[   2  202 -202]
  [   3  203 -203]
  [   4  204 -204]
  [   5  205 -205]
  [   6  206 -206]
  [1001  207 -207]]]
after 'prediction'
[[[   2  202 -202]
  [   3  203 -203]
  [   4  204 -204]
  [   5  205 -205]
  [   6  206 -206]
  [1001  207 -207]]] 1002

[[   3  203 -203]
 [   4  204 -204]
 [   5  205 -205]
 [   6  206 -206]
 [   7  207 -207]
 [1002  208 -208]] 1002


start loop 3
row
 [[[   3  203 -203]
  [   4  204 -204]
  [   5  205 -205]
  [   6  206 -206]
  [   7  207 -207]
  [1002  208 -208]]]
after 'prediction'
[[[   3  203 -203]
  [   4  204 -204]
  [   5  205 -205]
  [   6 

In [47]:
results = pd.DataFrame(data={"Actuals":target,
                            "Predictions":recur_preds})
results.head()

ValueError: array length 4 does not match index length 26